In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp
import yaml

import random
from itertools import chain, combinations, tee
import time


# Network Structure

In [2]:
class network_structure():
    def __init__(self, edges_nodes_matrix):
        # Structure of edges_nodes_matrix
        # An array of 2d-arrays, with each 2d-array encoding the start and end node indices of an edge.
        
        assert edges_nodes_matrix.shape[1] == 2, "We must have edges_nodes_matrix.shape[1] == 2"
        
        self.edges_nodes_matrix = edges_nodes_matrix
        self.nodes = np.unique(self.edges_nodes_matrix.flatten())
        self.edges = np.array(range(self.edges_nodes_matrix.shape[0]))
        self.num_nodes = len(self.nodes)
        self.num_edges = len(self.edges)
        
        self.construct_nodes_dict()
        self.construct_edges_dict()
#         self.construct_depth_dicts()
#         self.construct_height_dicts()
    
        origin_nodes = [node for node in self.nodes if self.nodes_dict[node]['incoming_edges'] == []]
        destination_nodes = [node for node in self.nodes if self.nodes_dict[node]['outgoing_edges'] == []]
        
        assert len(origin_nodes) == 1, "For now, we consider only single-origin \
            single-destination networks."
        assert len(destination_nodes) == 1, "For now, we consider only single-origin \
            single-destination networks."
        self.origin_node = origin_nodes[0]
        self.destination_node = destination_nodes[0]
        self.intermediate_nodes = [node for node in self.nodes if node not in [self.origin_node, self.destination_node]]
        
        self.start_edges = self.nodes_dict[self.origin_node]["outgoing_edges"]
        self.end_edges = self.nodes_dict[self.destination_node]["incoming_edges"]
        
        self.construct_routes_dict()
    
        return

    
    # Network structure functions:
    
    def construct_nodes_dict(self):
        self.nodes_dict = {}
        for node in self.nodes:
            self.nodes_dict[node] = {}
            self.nodes_dict[node]['incoming_edges'] = list((self.edges_nodes_matrix[:, 1] == node).nonzero()[0])
            self.nodes_dict[node]['outgoing_edges'] = list((self.edges_nodes_matrix[:, 0] == node).nonzero()[0])
            
            self.nodes_dict[node]['previous_nodes'] = list(set(self.edges_nodes_matrix[edge, 0] for edge in self.nodes_dict[node]['incoming_edges']))
            self.nodes_dict[node]['next_nodes'] = list(set(self.edges_nodes_matrix[edge, 1] for edge in self.nodes_dict[node]['outgoing_edges']))
        
        return

    def construct_edges_dict(self):
        self.edges_dict = {}

        for edge in self.edges:
            self.edges_dict[edge] = {}

        for node in self.nodes:
            if self.nodes_dict[node]['incoming_edges'] != []:
                for edge in self.nodes_dict[node]['incoming_edges']:
                    self.edges_dict[edge]['end_node'] = node

            if self.nodes_dict[node]['outgoing_edges'] != []:
                for edge in self.nodes_dict[node]['outgoing_edges']:
                    self.edges_dict[edge]['start_node'] = node

        for edge in self.edges:
            self.edges_dict[edge] = {}
            start_node = int(np.array([node for node in self.nodes if edge in self.nodes_dict[node]['outgoing_edges']]))
            end_node = int(np.array([node for node in self.nodes if edge in self.nodes_dict[node]['incoming_edges']]))
            self.edges_dict[edge]['start_node'] = start_node
            self.edges_dict[edge]['end_node'] = end_node
            self.edges_dict[edge]['incoming_edges'] = self.nodes_dict[start_node]['incoming_edges']
            self.edges_dict[edge]['outgoing_edges'] = self.nodes_dict[end_node]['outgoing_edges']
        return
    
    def construct_routes_dict(self):
        self.routes = [[edge] for edge in self.start_edges]
        
        route_construction_finished = False
        while route_construction_finished is False:
            route_construction_finished = True
            routes_temp = []
            for route_segment in self.routes:
                
#                 print()
#                 print("route_segment:", route_segment)
#                 print("route_segment[-1]:", route_segment[-1])
#                 print("self.edges_dict[route_segment[-1]]:", self.edges_dict[route_segment[-1]])
#                 print("self.edges_dict[route_segment[-1]][outgoing_edges]:", self.edges_dict[route_segment[-1]]["outgoing_edges"])
# #                 print()
                
                if self.edges_dict[route_segment[-1]]["outgoing_edges"] != []:
                    route_construction_finished = False
                    for edge in self.edges_dict[route_segment[-1]]["outgoing_edges"]:
                        routes_temp.append(route_segment + [edge])
                else:
                    routes_temp.append(route_segment)
                
#                 print("routes_temp:", routes_temp)
                
                
            if route_construction_finished is True:
                break
            self.routes = routes_temp
            
#             print()
#             print("self.routes:", self.routes)
#             print()

        return
    
    def print_nodes_dict(self):
        print()
        print("Printing nodes:")
        for node_key, node_value in self.nodes_dict.items():
            print()
            print("Node", node_key)
            for key, value in node_value.items():
                print(key, ":", value)
        print()
        return
    
    def print_edges_dict(self):
        print()
        print("Printing edges:")
        for edge_key, edge_value in self.edges_dict.items():
            print()
            print("Edge", edge_key)
            for key, value in edge_value.items():
                print(key, ":", value)
        print()
        return
    
    def print_routes(self):
        print()
        print("Printing routes:")
        print()
        for route_index, route_edges in enumerate(self.routes):
            print("Route", route_index, ":", route_edges)
        return
    
    def print_all(self):
        print()
        print("Printing network information:")
        print()
        print("Origin node:", self.origin_node)
        print("Destination node:", self.destination_node)
        print("Number of nodes:", self.num_nodes)
        print()
        print("Starting edges:", self.start_edges)
        print("Ending edges:", self.end_edges)
        print("Number of edges:", self.num_edges)
        
        self.print_nodes_dict()
        self.print_edges_dict()
        self.print_routes()
        
        return
    

In [3]:
# list_1 = [1, 2]
# list_2 = [3, 4, 5]
# list_1 = list_1 + [6]
# list_2.append(list_1)
# list_2

In [4]:
edges_nodes_matrix = np.array([[0, 1], [0, 2], [1, 2], [1, 3], [2, 3]])
netw_example = network_structure(edges_nodes_matrix)

# print("netw_example.edges:", netw_example.edges)
# print()
# print("netw_example.nodes:", netw_example.nodes)
# print()
# print("netw_example.edges_dict:", netw_example.edges_dict)
# print()
# print("netw_example.nodes_dict:", netw_example.nodes_dict)
# print()
# print("netw_example.routes:", netw_example.routes)
# print()

# netw_example.print_nodes_dict()
# netw_example.print_edges_dict()
# netw_example.print_routes()

netw_example.print_all()
# netw_example.intermediate_nodes



Printing network information:

Origin node: 0
Destination node: 3
Number of nodes: 4

Starting edges: [0, 1]
Ending edges: [3, 4]
Number of edges: 5

Printing nodes:

Node 0
incoming_edges : []
outgoing_edges : [0, 1]
previous_nodes : []
next_nodes : [1, 2]

Node 1
incoming_edges : [0]
outgoing_edges : [2, 3]
previous_nodes : [0]
next_nodes : [2, 3]

Node 2
incoming_edges : [1, 2]
outgoing_edges : [4]
previous_nodes : [0, 1]
next_nodes : [3]

Node 3
incoming_edges : [3, 4]
outgoing_edges : []
previous_nodes : [1, 2]
next_nodes : []


Printing edges:

Edge 0
start_node : 0
end_node : 1
incoming_edges : []
outgoing_edges : [2, 3]

Edge 1
start_node : 0
end_node : 2
incoming_edges : []
outgoing_edges : [4]

Edge 2
start_node : 1
end_node : 2
incoming_edges : [0]
outgoing_edges : [4]

Edge 3
start_node : 1
end_node : 3
incoming_edges : [0]
outgoing_edges : []

Edge 4
start_node : 2
end_node : 3
incoming_edges : [1, 2]
outgoing_edges : []


Printing routes:

Route 0 : [0, 2, 4]
Route 1 : [

# General CBCP Equilibrium Solver

## (Special Case) Quartic Polynomial Latency Functions

In [5]:
# grad = np.array([0, 1, 2, 3, 4])
grad = np.array([2, 4, 0, 1, 3])

for id_temp, entry_temp in enumerate(grad):
    print("id_temp, entry_temp:", id_temp, entry_temp)


id_temp, entry_temp: 0 2
id_temp, entry_temp: 1 4
id_temp, entry_temp: 2 0
id_temp, entry_temp: 3 1
id_temp, entry_temp: 4 3


In [6]:
def welfare_obj(T, network, lambda_E, lambda_R, lambda_I, tau, v_I_array, v_E_array, y_el, y_in, \
                a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):
    
    assert np.all(a_ex >= 0.0), "All entries of a_ex must be non-negative."
    assert np.all(a_gp >= 0.0), "All entries of a_gp must be non-negative."
    assert a_ex.shape == a_gp.shape, "a_ex and a_gp must have the same shape."
    assert len(a_ex.shape) in [1, 2], "a_ex and a_gp must be either a vector or a matrix."
    
    if len(a_ex.shape) == 1:
        latency_params_length = a_ex.shape[0]
        a_ex = a_ex.reshape((latency_params_length, 1)) @ np.ones((1, network.num_edges))
        a_gp = a_gp.reshape((latency_params_length, 1)) @ np.ones((1, network.num_edges))
    
    # In full:
    # y_el indices: (group, edge, "lane", time)
    # y_in indices: (group, edge, "lane", time)
    # Here:
    # y_el indices: (group, "lane")
    # y_in indices: (group, "lane")
    
    num_el = v_E_array.shape[0]
    num_in = v_I_array.shape[0]

    num_nodes = network.num_nodes
    num_edges = network.num_edges
    
#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert len(tau.shape) == 2, "tau should be 2-dimensional"
    assert tau.shape[0] == num_edges, "toll vector first axis length must equal the number of edges."
    assert tau.shape[1] == T, "toll vector second axis length must equal the number of edges."
    
    ## Compute lane flows:
    
    # Express lane (ex):
    x_ex = np.zeros((num_edges, T))
    for e in range(num_edges):
        for t in range(T):
            x_ex[e, t] += sum(y_el[(g, e, k, t)] for g in range(num_el) for k in [0, 1])
            x_ex[e, t] += sum(y_in[(g, e, 0, t)] for g in range(num_in))
    
    # General purpose lane (gp):
    x_gp = np.zeros(num_edges)
    for e in range(num_edges):
        for t in range(T):
            x_ex[e, t] += sum(y_el[(g, e, 2, t)] for g in range(num_el))
            x_ex[e, t] += sum(y_in[(g, e, 1, t)] for g in range(num_in))
    
    ell_ex = np.zeros((num_edges, T))
    ell_gp = np.zeros((num_edges, T))
    for e in range(num_edges):
        for t in range(T):
            ell_ex[e, t] = sum(a_ex[p] * (x_ex[e, t] ** p) for p in range(5))
            ell_gp[e, t] = sum(a_gp[p] * (x_gp[e, t] ** p) for p in range(5))
            
    # TODO: Edit below to incorporate time
    
    for e in range(num_edges):
        for t in range(T):
            obj_E = sum(tau[e, t] * y_el[(g, e, 1, t)] for g in range(num_el)) \
                        + sum(y_el[(g, e, k, t)] * v_E_array[g] * ell_ex[e] for g in range(num_el) for k in [0, 1]) \
                        + sum(y_el[(g, e, 2)] * v_E_array[g] * ell_gp[e] for g in range(num_el))
            obj_I = sum(tau[e] * y_in[(g, e, 0)] for g in range(num_in)) \
                        + sum(y_in[(g, e, 0)] * v_I_array[g] * ell_ex[e] for g in range(num_in)) \
                        + sum(y_in[(g, e, 1)] * v_I_array[g] * ell_gp[e] for g in range(num_in))
            obj_R = sum(tau[e] * y_el[(g, e, 1)] for g in range(num_el)) \
                        + sum(tau[e] * y_in[(g, e, 0)] for g in range(num_in))
    
    welfare = lambda_E * obj_E - lambda_R * obj_R + lambda_I * obj_I
    

#     print()
#     print("obj_E:", obj_E)
#     print("obj_R:", obj_R)
#     print("obj_I:", obj_I)
#     print("welfare:", welfare)
#     print()
    
    return welfare

def latency_total(network, tau, v_I_array, v_E_array, y_el, y_in, \
                  a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                  a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):
    
    assert np.all(a_ex >= 0.0), "All entries of a_ex must be non-negative."
    assert np.all(a_gp >= 0.0), "All entries of a_gp must be non-negative."
    assert a_ex.shape == a_gp.shape, "a_ex and a_gp must have the same shape."
    assert len(a_ex.shape) in [1, 2], "a_ex and a_gp must be either a vector or a matrix."
    
    if len(a_ex.shape) == 1:
        latency_params_length = a_ex.shape[0]
        a_ex = a_ex.reshape((latency_params_length, 1)) @ np.ones((1, network.num_edges))
        a_gp = a_gp.reshape((latency_params_length, 1)) @ np.ones((1, network.num_edges))
    
    # In full:
    # y_el indices: (group, edge, "lane", time)
    # y_in indices: (group, edge, "lane", time)
    # Here:
    # y_el indices: (group, "lane")
    # y_in indices: (group, "lane")
    
    num_el = v_E_array.shape[0]
    num_in = v_I_array.shape[0]

    num_nodes = network.num_nodes
    num_edges = network.num_edges
    
    ## Compute lane flows:
    
    # Express lane (ex):
    x_ex = np.zeros(num_edges)
    for e in range(num_edges):
        x_ex[e] += sum(y_el[(g, e, k)] for g in range(num_el) for k in [0, 1])
        x_ex[e] += sum(y_in[(g, e, 0)] for g in range(num_in))
    
    # General purpose lane (gp):
    x_gp = np.zeros(num_edges)
    for e in range(num_edges):
        x_ex[e] += sum(y_el[(g, e, 2)] for g in range(num_el))
        x_ex[e] += sum(y_in[(g, e, 1)] for g in range(num_in))
    
    latency_val = 0.0
    
    ell_ex = np.zeros(num_edges)
    ell_gp = np.zeros(num_edges)
    for e in range(num_edges):
        ell_ex[e] = sum(a_ex[p] * (x_ex[e] ** p) for p in range(5))
        
        ell_gp[e] = sum(a_gp[p] * (x_gp[e] ** p) for p in range(5))
        
        latency_val += x_ex[e] * ell_ex[e] + x_gp[e] * ell_gp[e]
    
    return latency_val



In [22]:
def proj_tau_B_11(network, tau, B, tau_max = 1.0, B_max = 1.0):
    
    num_edges = len(network.edges)
    num_nodes = len(network.nodes)
        
#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()
    
    assert tau.shape[0] == num_edges, "toll vector length must equal the number of edges."
    
    tau_feas = cp.Variable(num_edges)
    B_feas = cp.Variable(1)
    
    func = cp.sum_squares(tau_feas - tau) + (B_feas - B)**2

    objective = cp.Minimize(func)
        
    constraints = []
#     constraints += [0 <= tau_feas <= 1]
#     constraints += [0 <= B_feas <= 1]
    constraints += [tau_feas >= 0.0]
    constraints += [B_feas >= 0.0]
    constraints += [tau_feas <= tau_max * np.ones(num_edges)]
    constraints += [B_feas <= B_max]
    constraints += [B_feas <= sum(tau_feas[edge] for edge in route) for route in network.routes]
    
    prob = cp.Problem(objective, constraints)
    result = prob.solve()

    return np.round(np.maximum(tau_feas.value, 0.0), decimals=4), \
            np.round(np.maximum(B_feas.value, 0.0), decimals=4)


# Chinmay's Algorithm:

In [23]:
## Steps
# 1: Define variables
# 2: Define objective
# 3: Define constraints
# 4: Define problem
# 5: Solve problem
# 6: Extract values

In [24]:

# Below: For quartic latency functions:
# Latency Function: a_4 x^4 + a_3 x^3 + a_2 x^2 + a_1 x + a_0

def solve_CBCP_direct(network, tau, B, v_I_array, v_E_array, \
                         a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                         a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):
    
    assert np.all(a_ex >= 0.0), "All entries of a_ex must be non-negative."
    assert np.all(a_gp >= 0.0), "All entries of a_gp must be non-negative."
    assert a_ex.shape == a_gp.shape, "a_ex and a_gp must have the same shape."
    assert len(a_ex.shape) in [1, 2], "a_ex and a_gp must be either a vector or a matrix."
    
    if len(a_ex.shape) == 1:
        latency_params_length = a_ex.shape[0]
        a_ex = a_ex.reshape((latency_params_length, 1)) @ np.ones((1, network.num_edges))
        a_gp = a_gp.reshape((latency_params_length, 1)) @ np.ones((1, network.num_edges))
    
    # Variable indices:
    # In full:
    # y_el indices: (group, edge, "lane", time)
    # y_in indices: (group, edge, "lane", time)
    # Here:
    # y_el indices: (group, edge, "lane")
    # y_in indices: (group, edge, "lane")
    
    num_el = v_E_array.shape[0]
    num_in = v_I_array.shape[0]

    # Variables:
    y_el = {}
    for g in range(num_el):
        for e in network.edges:
            for k in [0, 1, 2]:
                y_el[(g, e, k)] = cp.Variable(1)
            
    y_in = {}
    for g in range(num_in):
        for e in network.edges:
            for k in [0, 1]:
                y_in[(g, e, k)] = cp.Variable(1)

    x_ex = cp.Variable(network.num_edges)
    x_gp = cp.Variable(network.num_edges)
    
    # Objective:
    func = 0.0
    for e in network.edges:
        func += 1/5 * a_ex[4] * cp.power(x_ex[e], 5)
        func += 1/4 * a_ex[3] * cp.power(x_ex[e], 4)
        func += 1/3 * a_ex[2] * cp.power(x_ex[e], 3)
        func += 1/2 * a_ex[1] * cp.power(x_ex[e], 2)
        func += a_ex[0] * x_ex[e]
        func += 1/5 * a_gp[4] * cp.power(x_gp[e], 5)
        func += 1/4 * a_gp[3] * cp.power(x_gp[e], 4)
        func += 1/3 * a_gp[2] * cp.power(x_gp[e], 3)
        func += 1/2 * a_gp[1] * cp.power(x_gp[e], 2)
        func += a_gp[0] * x_gp[e]
        
        func += sum(tau[e] * y_el[(g, e, 1)] / v_E_array[g] for g in range(num_el))
        func += sum(tau[e] * y_in[(g, e, 0)] / v_I_array[g] for g in range(num_in))

    objective = cp.Minimize(func)
    
    # Constraints:
    constraints = []
    
    constraints += [y_el[(g, e, k)] >= 0.0 for g in range(num_el) for e in network.edges for k in [0, 1, 2]]
    constraints += [y_in[(g, e, k)] >= 0.0 for g in range(num_in) for e in network.edges for k in [0, 1]]
        
    constraints += [sum(y_el[(g, e, k)] for e in network.start_edges for k in [0, 1, 2]) == 1.0 \
                    for g in range(num_el)]
    constraints += [sum(y_in[(g, e, k)] for e in network.start_edges for k in [0, 1]) == 1.0 \
                    for g in range(num_in)]
    for node in network.intermediate_nodes:
        constraints += [sum(y_el[(g, e, k)] for e in network.nodes_dict[node]["incoming_edges"] for k in [0, 1, 2]) \
                        == sum(y_el[(g, e, k)] for e in network.nodes_dict[node]["outgoing_edges"] for k in [0, 1, 2]) \
                        for g in range(num_el)]
        constraints += [sum(y_in[(g, e, k)] for e in network.nodes_dict[node]["incoming_edges"] for k in [0, 1]) \
                        == sum(y_in[(g, e, k)] for e in network.nodes_dict[node]["outgoing_edges"] for k in [0, 1]) \
                        for g in range(num_in)]
    
    for e in network.edges:
        constraints += [x_ex[e] == sum(y_el[(g, e, k)] for g in range(num_el) for k in [0, 1]) \
                                   + sum(y_in[(g, e, 0)] for g in range(num_in)) ]
        constraints += [x_gp[e] == sum(y_el[(g, e, 2)] for g in range(num_el)) \
                                   + sum(y_in[(g, e, 1)] for g in range(num_in)) ]
    
    constraints += [sum(y_el[(g, e, 0)]  * tau[e] for e in network.edges) <= B for g in range(num_el)]
    
    # Problem:
    prob = cp.Problem(objective, constraints)
    
    # Solve:
    result = prob.solve()
    
    # Extract Values:
    y_el_values = {}
    for g in range(num_el):
        for e in network.edges:
            for k in [0, 1, 2]:
                y_el_values[(g, e, k)] = max(y_el[(g, e, k)].value[0], 0.0)

    y_in_values = {}
    for g in range(num_in):
        for e in network.edges:
            for k in [0, 1]:
                y_in_values[(g, e, k)] = max(y_in[(g, e, k)].value[0], 0.0)

    return y_el_values, y_in_values


In [25]:
time_1 = time.time()

edges_nodes_matrix = np.array([[0, 1], [0, 2], [1, 2], [1, 3], [2, 3]])
network_Braess = network_structure(edges_nodes_matrix)
# network_Braess.print_all()

network = network_Braess

tau = np.array([0.2, 0.6, 0.5, 0.8, 0.4])
B = 0.8
v_I_array = np.array([1.0, 1.2, 1.4])
v_E_array = np.array([0.6, 0.8, 0.9])
flow_per_group = 1.0
a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])
num_iters_max = 5000
error_bound = 1E-3
diffs_num_cols = 5
# y_init = np.array([0.0, 0.05, 0.95, 0.95, 0.05])

# lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0
lambda_E, lambda_R, lambda_I = 10.0, 0.2, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0

# tau_max, B_max = 1.0, 1.0
flow_max = flow_per_group * (v_I_array.shape[0] + v_E_array.shape[0])
tau_max = sum(a[k] * (flow_max ** k) for k in range(5))
B_max = tau_max

d = 2
num_iters = 1000
tau = np.zeros((network.num_edges, num_iters))
tau_perturbed = np.zeros((network.num_edges, num_iters))

B = np.zeros(num_iters)
B_perturbed = np.zeros(num_iters)
delta = np.zeros(num_iters)
eta = np.zeros(num_iters)
eta_bar = 3.0
delta_bar = 3.0

welfare_list = []

# tau[:, 0] = tau_max * 0.8
# B[0] = B_max * 0.2

tau[:, 0] = tau_max * 0.7
B[0] = B_max * 0.3

# tau[:, 0] = tau_max * 0.6
# B[0] = B_max * 0.4


for i in range(num_iters-1):
    
    print()
    print("Iter:", i)
    
    eta[i] = eta_bar * (i+1)**(-1/2) * d**(-1)
    delta[i] = delta_bar * (i+1)**(-1/4) * d**(-1/2)
    w_i_unnormalized = np.random.randn(network.num_edges +1)
    w_i = w_i_unnormalized / np.linalg.norm(w_i_unnormalized)
    print("w_i:", w_i)
    tau_perturbed[:, i] = tau[:, i] + delta[i] * w_i[:-1]
    B_perturbed[i] = B[i] + delta[i] * w_i[-1]
    
#     if tau_perturbed[:, i] < B_perturbed[i] or tau_perturbed[:, i] < 0 or B_perturbed[i] < 0:
    tau_perturbed[:, i], B_perturbed[i] = proj_tau_B_11(network, tau_perturbed[:, i], B_perturbed[i], \
                                                    tau_max = tau_max, B_max = B_max)
    
    print("tau[:, i]:", tau[:, i])
    print("B[i]:", B[i])
    print("tau_perturbed[:, i]:", tau_perturbed[:, i])
    print("B_perturbed[i]:", B_perturbed[i])


    y_el_values, y_in_values = solve_CBCP_direct(network, tau = tau[:, i], B = B[i], v_I_array = v_I_array, \
                                                 v_E_array = v_E_array, a_ex = a, a_gp = a)

#     print()
#     print("y_el_values:", y_el_values)
#     print("y_in_values:", y_in_values)
#     print()

#     print()
#     print("New solve_CBCP_iter_11 call to solve_CBCP (perturbed):")
#     print()

    y_el_perturbed_values, y_in_perturbed_values = solve_CBCP_direct(network, tau = tau_perturbed[:, i], B = B_perturbed[i], \
                                                                     v_I_array = v_I_array, v_E_array = v_E_array, a_ex = a, a_gp = a)
    
#     print("y_el:", y_el)
#     print("y_in:", y_in)
#     print("y_el_perturbed:", y_el_perturbed)
#     print("y_in_perturbed:", y_in_perturbed)
    
    welfare = welfare_obj(network, lambda_E, lambda_R, lambda_I, tau = tau[:, i], v_I_array = v_I_array, v_E_array = v_E_array, \
                          y_el = y_el_values, y_in = y_in_values, a_ex = a, a_gp = a)
    welfare_perturbed = welfare_obj(network, lambda_E, lambda_R, lambda_I, tau = tau_perturbed[:, i], \
                                    v_I_array = v_I_array, v_E_array = v_E_array, \
                                    y_el = y_el_perturbed_values, y_in = y_in_perturbed_values, a_ex = a, a_gp = a)
    welfare_list.append(welfare)
    
#     print("welfare:", welfare)
#     print("welfare_perturbed:", welfare_perturbed)
#     print("tau[:, i] - eta[i] * (d/delta[i]) * w_i[:-1] * (welfare_perturbed - welfare):\n", tau[:, i] - eta[i] * (d/delta[i]) * w_i[:-1] * (welfare_perturbed - welfare))
    
    tau[:, i+1] = tau[:, i] - eta[i] * (d/delta[i]) * w_i[:-1] * (welfare_perturbed - welfare)
    
#     print()
#     print("B[i] - eta[i] * (d/delta[i]) * w_i[-1] * (welfare_perturbed - welfare):\n", \
#          B[i] - eta[i] * (d/delta[i]) * w_i[-1] * (welfare_perturbed - welfare))
#     print("B[i]:", B[i])
#     print("eta[i]:", eta[i])
#     print("delta[i]:", delta[i])
#     print("w_i[-1]:", w_i[-1])
#     print("welfare:", welfare)
#     print("welfare_perturbed:", welfare_perturbed)
#     print()
    
    B[i+1] = B[i] - eta[i] * (d/delta[i]) * w_i[-1] * (welfare_perturbed - welfare)
    tau[:, i+1], B[i+1] = proj_tau_B_11(network, tau[:, i+1], B[i+1], tau_max, B_max)
    
    if i >= diffs_num_cols + 2:
        tau_diffs = np.linalg.norm(tau[:, i-diffs_num_cols : i-1] - tau[:, i-diffs_num_cols+1 : i], axis = 0)
        B_diffs = B[i-diffs_num_cols : i-1] - B[i-diffs_num_cols+1 : i]
        
#         print("tau[:, 0:10]:", tau[:, 0:10])
#         print("B[0:10]:", B[0:10])
        print("tau_diffs:", tau_diffs)
        print("B_diffs:", B_diffs)
        
        if max(np.max(np.absolute(tau_diffs)), np.max(np.absolute(B_diffs))) < error_bound:
            break

time_2 = time.time()

min_welfare = min(welfare_list)
argmin_welfare_list = welfare_list.index(min(welfare_list))
argmin_tau = tau[:, argmin_welfare_list]
argmin_B = B[argmin_welfare_list]

print()
print("Time:", time_2 - time_1)




Iter: 0
w_i: [-0.26747571  0.3426618  -0.77909342 -0.44482991 -0.07829717 -0.00699965]
tau[:, i]: [4.2 4.2 4.2 4.2 4.2]
B[i]: 1.7999999999999998
tau_perturbed[:, i]: [3.6326 4.9269 2.5473 3.2564 4.0339]
B_perturbed[i]: 1.7852

Iter: 1
w_i: [-0.60909301 -0.45334105 -0.32060424  0.45663025  0.24772108 -0.22544091]
tau[:, i]: [6. 0. 6. 6. 6.]
B[i]: 3.5929
tau_perturbed[:, i]: [4.9135 0.     5.4281 6.     6.    ]
B_perturbed[i]: 3.1908

Iter: 2
w_i: [-0.22477669 -0.13227349  0.06855454 -0.6725796   0.6444642   0.24409428]
tau[:, i]: [0. 0. 0. 6. 6.]
B[i]: 0.0
tau_perturbed[:, i]: [0.     0.     0.1105 4.9159 6.    ]
B_perturbed[i]: 0.3934

Iter: 3
w_i: [ 0.47026114  0.33324518 -0.37644084 -0.32752664  0.41659244  0.49524887]
tau[:, i]: [6. 6. 0. 6. 0.]
B[i]: 0.0
tau_perturbed[:, i]: [6.     6.     0.     5.5087 0.6249]
B_perturbed[i]: 0.7429

Iter: 4
w_i: [ 0.17788069  0.24875863  0.25380242  0.24818179 -0.62356764  0.62580438]
tau[:, i]: [6. 6. 0. 0. 6.]
B[i]: 6.0
tau_perturbed[:, i]: [6

tau_diffs: [ 8.48528137 10.39230485  8.48528137 10.39230485]
B_diffs: [ 6.  0. -6.  6.]

Iter: 30
w_i: [ 0.12373184 -0.37098155  0.75259087 -0.46865671  0.14287881 -0.2015357 ]
tau[:, i]: [6. 6. 0. 0. 6.]
B[i]: 0.0
tau_perturbed[:, i]: [6.     5.6665 0.6766 0.     6.    ]
B_perturbed[i]: 0.0
tau_diffs: [10.39230485  8.48528137 10.39230485 10.39230485]
B_diffs: [ 0. -6.  6.  0.]

Iter: 31
w_i: [ 1.26613784e-01  3.75286775e-01 -5.89790015e-04  5.97612286e-01
  3.67953792e-01  5.92113165e-01]
tau[:, i]: [6.     4.9935 2.0419 0.     6.    ]
B[i]: 0.0
tau_perturbed[:, i]: [6.     5.3282 2.0414 0.533  6.    ]
B_perturbed[i]: 0.5281
tau_diffs: [ 8.48528137 10.39230485 10.39230485 12.        ]
B_diffs: [-6.  6.  0.  0.]

Iter: 32
w_i: [ 0.39057589  0.38295096 -0.80361487 -0.04356876 -0.03130181 -0.22830713]
tau[:, i]: [6.     6.     2.0376 4.3095 6.    ]
B[i]: 4.2699
tau_perturbed[:, i]: [6.     6.     1.3263 4.2709 5.9723]
B_perturbed[i]: 4.0678
tau_diffs: [10.39230485 10.39230485 12.        

tau_diffs: [10.39230485  0.          6.17938343  6.17938343]
B_diffs: [-6.  6. -6.  6.]

Iter: 57
w_i: [-0.25527834 -0.39640342  0.85028677 -0.10245926  0.20470312 -0.0480462 ]
tau[:, i]: [6. 6. 0. 0. 0.]
B[i]: 6.0
tau_perturbed[:, i]: [5.8773 5.7076 0.6536 0.     0.1697]
B_perturbed[i]: 5.8773
tau_diffs: [0.         6.17938343 6.17938343 8.49745027]
B_diffs: [ 6. -6.  6. -6.]

Iter: 58
w_i: [-0.27052328  0.92086099  0.18483901 -0.14789876  0.11525876 -0.09750953]
tau[:, i]: [6. 6. 0. 6. 0.]
B[i]: 6.0
tau_perturbed[:, i]: [5.7929 6.     0.1415 5.8868 0.0882]
B_perturbed[i]: 5.9254
tau_diffs: [6.17938343 6.17938343 8.49745027 5.5454    ]
B_diffs: [-6.  6. -6.  0.]

Iter: 59
w_i: [ 0.04465183 -0.15847557 -0.70463009  0.27002905  0.63380943  0.04192932]
tau[:, i]: [6.     0.     0.     6.     1.6657]
B[i]: 1.6657
tau_perturbed[:, i]: [6.     0.     0.     6.     2.1488]
B_perturbed[i]: 1.6977
tau_diffs: [6.17938343 8.49745027 5.5454     6.        ]
B_diffs: [ 6. -6.  0.  0.]

Iter: 60
w_i

tau_diffs: [ 6.65846207 12.29855291  8.0676939   4.06034704]
B_diffs: [2.9436 0.     0.     0.    ]

Iter: 84
w_i: [-0.37450264  0.29708354 -0.12770369  0.15412106 -0.22780048 -0.82433886]
tau[:, i]: [6.     2.5016 4.3608 6.     6.    ]
B[i]: 3.3569
tau_perturbed[:, i]: [5.7384 2.7092 4.2716 6.     5.8409]
B_perturbed[i]: 2.781
tau_diffs: [12.29855291  8.0676939   4.06034704  9.38713658]
B_diffs: [ 0.      0.      0.     -3.5415]

Iter: 85
w_i: [ 0.1733054   0.22286295  0.47467686 -0.40677716  0.5322215  -0.49623769]
tau[:, i]: [0. 6. 0. 6. 0.]
B[i]: 0.0
tau_perturbed[:, i]: [0.1207 6.     0.3307 5.7166 0.3707]
B_perturbed[i]: 0.0
tau_diffs: [8.0676939  4.06034704 9.38713658 8.2009377 ]
B_diffs: [ 0.      0.     -3.5415  0.1846]

Iter: 86
w_i: [-0.16422886  0.27628981 -0.22025726 -0.0468357  -0.91929061 -0.02984552]
tau[:, i]: [6. 6. 6. 0. 6.]
B[i]: 0.0
tau_perturbed[:, i]: [5.8859 6.     5.847  0.     5.3615]
B_perturbed[i]: 0.0
tau_diffs: [ 4.06034704  9.38713658  8.2009377  10.16146

tau_diffs: [10.39230485 10.39230485 10.39230485 10.39230485]
B_diffs: [-6.  6.  0.  0.]

Iter: 111
w_i: [-0.353665   -0.40412713 -0.62276491 -0.09707386 -0.51653427  0.21802572]
tau[:, i]: [1.0502 4.6826 6.     0.     0.    ]
B[i]: 0.0
tau_perturbed[:, i]: [0.8196 4.4191 5.5939 0.     0.    ]
B_perturbed[i]: 0.1422
tau_diffs: [10.39230485 10.39230485 10.39230485  0.33767868]
B_diffs: [ 6.0e+00  0.0e+00  0.0e+00 -4.4e-03]

Iter: 112
w_i: [-0.27966414 -0.13813458 -0.4434072  -0.09809016  0.7439035   0.37826283]
tau[:, i]: [6. 6. 6. 6. 6.]
B[i]: 0.0
tau_perturbed[:, i]: [5.818  5.9101 5.7115 5.9362 6.    ]
B_perturbed[i]: 0.2461
tau_diffs: [10.39230485 10.39230485  0.33767868  7.77784641]
B_diffs: [ 0.      0.     -0.0044  0.0044]

Iter: 113
w_i: [ 0.41258595  0.33176338 -0.12667148 -0.41592375 -0.32728803 -0.65080736]
tau[:, i]: [6. 6. 6. 6. 0.]
B[i]: 0.0
tau_perturbed[:, i]: [6.     6.     5.9178 5.73   0.    ]
B_perturbed[i]: 0.0
tau_diffs: [10.39230485  0.33767868  7.77784641  9.91141

...

tau_diffs: [8.50113881 0.14896046 0.50158471 0.19885072]
B_diffs: [ 0.     -0.0568  0.0568  0.    ]

Iter: 894
w_i: [-0.60405466  0.02123695  0.16141434  0.41705823 -0.33044623 -0.57050863]
tau[:, i]: [5.9855 5.1039 3.4265 6.     4.8966]
B[i]: 0.1428
tau_perturbed[:, i]: [5.7512 5.1121 3.4891 6.     4.7684]
B_perturbed[i]: 0.0
tau_diffs: [0.14896046 0.50158471 0.19885072 3.85549785]
B_diffs: [-0.0568  0.0568  0.      0.    ]

Iter: 895
w_i: [ 0.05002336  0.27546059 -0.25118117 -0.62929199 -0.67042215 -0.1142492 ]
tau[:, i]: [0.     5.3683 5.4358 6.     0.7833]
B[i]: 0.0
tau_perturbed[:, i]: [0.0194 5.4751 5.3384 5.756  0.5234]
B_perturbed[i]: 0.0
tau_diffs: [0.50158471 0.19885072 3.85549785 0.20422331]
B_diffs: [ 0.0568  0.      0.     -0.1428]

Iter: 896
w_i: [-0.14208059  0.07815848  0.73879077  0.20770527  0.54129408 -0.30290558]
tau[:, i]: [6. 6. 0. 0. 0.]
B[i]: 0.0
tau_perturbed[:, i]: [5.9449 6.     0.2864 0.0805 0.2098]
B_perturbed[i]: 0.0
tau_diffs: [0.19885072 3.85549785 0.204

tau_diffs: [7.4698691  6.33314435 5.81275721 1.69394835]
B_diffs: [ 0.     -2.7594 -3.2406  0.2789]

Iter: 921
w_i: [-0.58688826  0.28881609 -0.30737754  0.59189428 -0.24911621  0.25547751]
tau[:, i]: [6.     5.9878 6.     6.     5.9879]
B[i]: 0.0082
tau_perturbed[:, i]: [5.7741 6.     5.8817 6.     5.892 ]
B_perturbed[i]: 0.1066
tau_diffs: [6.33314435 5.81275721 1.69394835 4.45754149]
B_diffs: [-2.7594 -3.2406  0.2789  5.7211]

Iter: 922
w_i: [ 0.17005487  0.78921648  0.10161847 -0.14656437 -0.2435373  -0.50705112]
tau[:, i]: [6.     3.943  6.     1.8093 6.    ]
B[i]: 0.0
tau_perturbed[:, i]: [6.     4.2467 6.     1.7529 5.9063]
B_perturbed[i]: 0.0
tau_diffs: [5.81275721 1.69394835 4.45754149 0.01718284]
B_diffs: [-3.2406  0.2789  5.7211 -0.0082]

Iter: 923
w_i: [ 0.54049748  0.09832909 -0.11899728  0.54384458 -0.012908    0.6229767 ]
tau[:, i]: [6.     3.9899 6.     1.8006 5.9855]
B[i]: 0.0
tau_perturbed[:, i]: [6.     4.0277 5.9542 2.0098 5.9805]
B_perturbed[i]: 0.2397
tau_diffs: [1

tau_diffs: [ 7.00878029 10.39230485 12.          8.48577638]
B_diffs: [-2.7291  6.      0.     -5.9997]

Iter: 947
w_i: [ 0.57138175  0.0285573  -0.66312953  0.22381509  0.3888314  -0.17799873]
tau[:, i]: [6.     5.3049 0.     6.     6.    ]
B[i]: 6.0
tau_perturbed[:, i]: [6.     5.3158 0.     6.     6.    ]
B_perturbed[i]: 5.932
tau_diffs: [10.39230485 12.          8.48577638  3.22483154]
B_diffs: [ 6.      0.     -5.9997  5.9997]

Iter: 948
w_i: [ 0.23600183  0.70595394 -0.30588416 -0.47544315  0.02642721  0.3544326 ]
tau[:, i]: [6.     5.4517 0.     6.     6.    ]
B[i]: 5.085
tau_perturbed[:, i]: [6.     5.7215 0.     5.8183 6.    ]
B_perturbed[i]: 5.2205
tau_diffs: [12.          8.48577638  3.22483154  6.70950657]
B_diffs: [ 0.     -5.9997  5.9997 -6.    ]

Iter: 949
w_i: [ 0.91408471  0.00412903 -0.15535483 -0.08663756 -0.18994092 -0.31098771]
tau[:, i]: [5.9096 5.1812 0.1172 6.     5.9899]
B[i]: 4.9492
tau_perturbed[:, i]: [6.     5.1828 0.0578 5.9669 5.9173]
B_perturbed[i]: 4.83

tau_diffs: [ 0.04915455 11.3661033   8.70594827  5.57844572]
B_diffs: [0.0346 0.0511 0.     0.    ]

Iter: 973
w_i: [-0.20224812  0.12533619  0.06962864  0.60765404  0.73467659  0.17188737]
tau[:, i]: [6.     0.     0.     1.3076 6.    ]
B[i]: 0.0
tau_perturbed[:, i]: [5.9232 0.0476 0.0264 1.5383 6.    ]
B_perturbed[i]: 0.0653
tau_diffs: [11.3661033   8.70594827  5.57844572  2.18328226]
B_diffs: [0.0511 0.     0.     0.    ]

Iter: 974
w_i: [-0.0984094   0.08090503 -0.06738636 -0.35134933 -0.66149745  0.6466868 ]
tau[:, i]: [0.1218 3.6428 2.0237 6.     6.    ]
B[i]: 4.9957
tau_perturbed[:, i]: [0.0844 3.6735 1.9981 5.8666 5.7489]
B_perturbed[i]: 5.2412
tau_diffs: [8.70594827 5.57844572 2.18328226 8.5854422 ]
B_diffs: [0. 0. 0. 0.]

Iter: 975
w_i: [-0.173542   -0.18177459 -0.4322625  -0.40078513 -0.76606094 -0.05012214]
tau[:, i]: [1.8635 2.2109 3.2163 6.     6.    ]
B[i]: 0.0
tau_perturbed[:, i]: [1.7976 2.1419 3.0522 5.8479 5.7093]
B_perturbed[i]: 0.0
tau_diffs: [5.57844572 2.18328226

tau_diffs: [4.87933442 3.67482995 5.56735539 9.82782213]
B_diffs: [ 0.     -1.7107  1.7107  0.    ]

Time: 327.51626205444336


In [26]:
print("first(welfare_list):", welfare_list[0])
print("min(welfare_list):", min(welfare_list))
print("max(welfare_list):", max(welfare_list))
print("argmin_tau:", argmin_tau)
print("argmin_B:", argmin_B)

first(welfare_list): 1241.3822945733243
min(welfare_list): 537.305998709307
max(welfare_list): 3333.0055027031794
argmin_tau: [6. 6. 0. 6. 6.]
argmin_B: 0.0


In [27]:
y_el_values

{(0, 0, 0): 0.0,
 (0, 0, 1): -9.502683639972531e-09,
 (0, 0, 2): 0.493457714485248,
 (0, 1, 0): 0.0,
 (0, 1, 1): 1.7970120319180709e-09,
 (0, 1, 2): 0.5065422932204237,
 (0, 2, 0): 0.0,
 (0, 2, 1): -8.84933621627251e-10,
 (0, 2, 2): 9.300088893194098e-09,
 (0, 3, 0): 0.0,
 (0, 3, 1): -7.739531210116152e-09,
 (0, 3, 2): 0.49345770430694036,
 (0, 4, 0): 0.0,
 (0, 4, 1): 1.1932470206122187e-08,
 (0, 4, 2): 0.5065422915001208,
 (1, 0, 0): 0.0,
 (1, 0, 1): 3.6040274098425584e-09,
 (1, 0, 2): 0.49734517393691074,
 (1, 1, 0): 0.0,
 (1, 1, 1): 7.865103515537742e-09,
 (1, 1, 2): 0.5026548145939583,
 (1, 2, 0): 0.0,
 (1, 2, 1): -3.738910440222777e-10,
 (1, 2, 2): 9.301618939006658e-09,
 (1, 3, 0): 0.0,
 (1, 3, 1): 8.864202594160986e-09,
 (1, 3, 2): 0.4973451597490076,
 (1, 4, 0): 0.0,
 (1, 4, 1): 5.390190072063206e-09,
 (1, 4, 2): 0.5026548259965996,
 (2, 0, 0): 0.0,
 (2, 0, 1): 6.392217200001937e-09,
 (2, 0, 2): 0.5016743680209568,
 (2, 1, 0): 0.0,
 (2, 1, 1): 2.0003635488534995e-08,
 (2, 1, 2)

In [13]:
y_in_values

{(0, 0, 0): 0.5821719216319861,
 (0, 0, 1): 0.2094961277106064,
 (0, 1, 0): 0.004180742490241613,
 (0, 1, 1): 0.20415120816716587,
 (0, 2, 0): 4.144949098608525e-09,
 (0, 2, 1): 1.1425231528975375e-07,
 (0, 3, 0): 0.5846215247840308,
 (0, 3, 1): 0.20704640616129735,
 (0, 4, 0): 5.016818349357077e-07,
 (0, 4, 1): 0.20833156737283695,
 (1, 0, 0): 0.7108227707122439,
 (1, 0, 1): 4.161077085583072e-09,
 (1, 1, 0): 0.28917722088739817,
 (1, 1, 1): 4.239280934608319e-09,
 (1, 2, 0): 5.664237206604805e-09,
 (1, 2, 1): 1.505274957584457e-07,
 (1, 3, 0): 0.7108226144311154,
 (1, 3, 1): 4.250472527391911e-09,
 (1, 4, 0): 0.28917737714979724,
 (1, 4, 1): 4.168614768110095e-09,
 (2, 0, 0): 1.5326673968831134e-06,
 (2, 0, 1): 1.8587508398815776e-09,
 (2, 1, 0): 0.9999984635749538,
 (2, 1, 1): 1.8988984661208134e-09,
 (2, 2, 0): 7.3891652192127915e-09,
 (2, 2, 1): 2.2095957174396205e-07,
 (2, 3, 0): 1.3042789283025461e-06,
 (2, 3, 1): 1.89848245727403e-09,
 (2, 4, 0): 0.9999986919690309,
 (2, 4, 1):

## Grid Search:

In [ ]:
# tau = 0.5
# B = 0.5
# v_I = 1.0
# v_E = 0.6
# a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])

# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0

# y_el_point_5, y_in_point_5 = solve_CBCP_direct(tau = tau, B = B, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]),\
#                                                a_ex = a, a_gp = a)

# welfare_obj_arr[tau_index][B_index] = welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]), \
#                                                   y_el = y_el_point_5, y_in = y_in_point_5, \
#                                                   a_ex = a, a_gp = a)


In [ ]:
time_1 = time.time()

tau = 0.4
B = 0.3
v_I_array = np.array([1.0])
v_E_array = np.array([0.6])
a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])
num_iters_max = 5000
error_bound = 1E-3
diffs_num_cols = 5
y_init = np.array([0.0, 0.0, 1.0, 1.0, 0.0])
lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 0.2, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0


grid_size = 0.02
tau_arr = (np.arange(int(1/grid_size)) + 1) * grid_size
B_arr = (np.arange(int(1/grid_size)) + 1) * grid_size

welfare_obj_arr = np.ones((tau_arr.shape[0], B_arr.shape[0])) * 100
y_el_arr = np.zeros((tau_arr.shape[0], B_arr.shape[0], 3))
y_in_arr = np.zeros((tau_arr.shape[0], B_arr.shape[0], 2))
welfare_obj_list = []

for tau_index, tau in enumerate(tau_arr):
    for B_index, B in enumerate(B_arr):
        if B < tau:
            print("tau:", tau)
            print("B:", B)
            y_el_arr[tau_index, B_index, :], y_in_arr[tau_index, B_index, :] \
                = solve_CBCP_direct(tau = tau, B = B,\
                                   v_I_array = v_I_array, v_E_array = v_E_array, a_ex = a, a_gp = a)
            
#             def solve_CBCP_direct(tau, B, v_I, v_E, a = np.array([0.5, 1.0])):
            
            welfare_obj_arr[tau_index][B_index] = welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I_array = v_I_array, v_E_array = v_E_array, \
                                                              y_el = y_el_arr[tau_index, B_index, :], y_in = y_in_arr[tau_index, B_index, :], \
                                                              a_ex = a, a_gp = a)
        
            welfare_obj_list.append(welfare_obj_arr[tau_index][B_index])
            
            print()

time_2 = time.time()

print("Time:", time_2 - time_1)



In [ ]:
# tau = 0.5
# B = 0.4
# v_I = 1.0
# v_E = 0.6

# y_el_arr[tau_index, B_index, :], y_in_arr[tau_index, B_index, :] \
#                 = solve_CBCP_direct(tau = tau, B = B, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]), a_ex = a, a_gp = a)
            
# #             def solve_CBCP_direct(tau, B, v_I, v_E, a = np.array([0.5, 1.0])):
            
# welfare_obj_arr[tau_index][B_index] = welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]), \
#                                                               y_el = y_el_arr[tau_index, B_index, :], y_in = y_in_arr[tau_index, B_index, :], \
#                                                               a_ex = a, a_gp = a)

In [ ]:
# welfare_obj_arr

argmin_indices_wrapped = np.where(welfare_obj_arr == min(welfare_obj_list))
argmin_indices = [argmin_indices_wrapped[0][0], argmin_indices_wrapped[1][0]]
# argmin_indices
argmin_tau = tau_arr[argmin_indices[0]]
argmin_B = B_arr[argmin_indices[1]]

print("argmin_tau:\n", np.round(argmin_tau, 2))
print("\nargmin_B:\n", np.round(argmin_B, 2))

welfare_obj_arr[argmin_indices[0], argmin_indices[1]]


In [ ]:
# lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0

# plt.imshow(welfare_obj_arr.T, vmin = min(welfare_obj_list), vmax = max(welfare_obj_list), origin='lower') 
plt.imshow(welfare_obj_arr.T, extent=[np.min(tau_arr), np.max(tau_arr), np.min(B_arr), np.max(B_arr)], \
           vmin = min(welfare_obj_list), vmax = max(welfare_obj_list), origin='lower') 

plt.colorbar() 
plt.xlabel("Toll")
plt.ylabel("Budget")
# plt.xticks(x_positions, x_labels)

## Test:

In [ ]:
# Test:

grad = np.array([3.11430535, 1.501, 1.501, 2.46858321, 1.501])

# y_el: \hat y_1 E, \tilde y_1 E, y_2 E
y_el_var = cp.Variable(3)
# y_in: y_1 I, y_2 I
y_in_var = cp.Variable(2)

objective = cp.Minimize(grad[0:3] @ y_el_var + grad[3:] @ y_in_var)

constraints = []
constraints += [y_el_var >= 0, y_in_var >= 0]
constraints += [cp.sum(y_el_var) == 1.0, cp.sum(y_in_var) == 1.0]
constraints += [y_el_var[1] * tau <= B]

prob = cp.Problem(objective, constraints)
result = prob.solve()

print("grad:", grad)
print("y_el_var.value:", y_el_var.value)
# print("y_el_var_current:", y_el_var_current)
print("y_in_var.value:", y_in_var.value)
# print("y_in_var_current:", y_in_var_current)
print()

# y_el_var_current = y_el_var_current + 2/(k+2) * (y_el_var.value - y_el_var_current)
# y_in_var_current = y_in_var_current + 2/(k+2) * (y_in_var.value - y_in_var_current)

# y_iters[0:3, k] = y_el_var_current
# y_iters[3:, k] = y_in_var_current

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()

# Solver=SCS,verbose=False

In [ ]:
# y_el_var
# y_in_var
# np.hstack((y_el_var, y_in_var))

# Unused Functions for Network:

In [ ]:

def construct_depth_dicts(self):
    self.depth_edges_dict = {}
    counter = 1

    edges_remaining = list(self.edges)
    edges_accounted = []
#         print("[edge for edge in edges_remaining]:", [edge for edge in edges_remaining])
    edges_temp = [edge for edge in edges_remaining if len(self.edges_dict[edge]['incoming_edges']) == 0]
    self.depth_edges_dict[1] = edges_temp
    edges_remaining = [edge for edge in edges_remaining if edge not in edges_temp]
    edges_accounted.extend(edges_temp)
    counter += 1

    while edges_remaining != []:
#             print("[edge for edge in edges_remaining]:", [edge for edge in edges_remaining])
        edges_temp = [edge for edge in edges_remaining if set(self.edges_dict[edge]['incoming_edges']).issubset(set(edges_accounted))]
        self.depth_edges_dict[counter] = edges_temp
        edges_remaining = [edge for edge in edges_remaining if edge not in edges_temp]
        edges_accounted.extend(edges_temp)
        counter += 1

    self.depths = list(self.depth_edges_dict.keys())

    self.depth_nodes_dict = {}
    for depth in self.depths:
        nodes_temp = np.array([self.edges_dict[edge]['start_node'] for edge in self.depth_edges_dict[depth]])
        nodes_temp = list(np.unique(nodes_temp.flatten()))
        self.depth_nodes_dict[depth] = nodes_temp

    return

def construct_height_dicts(self):
    self.height_edges_dict = {}
    counter = 1

    edges_remaining = list(self.edges)
    edges_accounted = []
#         print("[edge for edge in edges_remaining]:", [edge for edge in edges_remaining])
    edges_temp = [edge for edge in edges_remaining if len(self.edges_dict[edge]['outgoing_edges']) == 0]
    self.height_edges_dict[1] = edges_temp
    edges_remaining = [edge for edge in edges_remaining if edge not in edges_temp]
    edges_accounted.extend(edges_temp)
    counter += 1

    while edges_remaining != []:
#             print("[edge for edge in edges_remaining]:", [edge for edge in edges_remaining])
        edges_temp = [edge for edge in edges_remaining if set(self.edges_dict[edge]['outgoing_edges']).issubset(set(edges_accounted))]
        self.height_edges_dict[counter] = edges_temp
        edges_remaining = [edge for edge in edges_remaining if edge not in edges_temp]
        edges_accounted.extend(edges_temp)
        counter += 1

    self.heights = list(self.height_edges_dict.keys())

    self.height_nodes_dict = {}
    for height in self.heights:
        nodes_temp = np.array([self.edges_dict[edge]['end_node'] for edge in self.height_edges_dict[height]])
        nodes_temp = list(np.unique(nodes_temp.flatten()))
        self.height_nodes_dict[height] = nodes_temp

    return

# Scratch Work:

In [ ]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()


In [ ]:
# # Test:

# Variables:

# y_el: \hat y_1 E, \tilde y_1 E, y_2 E
y_elig = cp.Variable(3)
# y_in: y_1 I, y_2 I
y_inel = cp.Variable(2)


# Objective:

a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])

tau = 0.5
B = 0.4

func = 0.0
func += 1/5 * a[4] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 5)
func += 1/4 * a[3] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 4)
func += 1/3 * a[2] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 3)
func += 1/2 * a[1] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 2)
func += a[0] * (y_elig[0] + y_elig[1] + y_inel[0])
func += y_inel[0] * tau / v_I + y_elig[0] * tau / v_E
func += 1/5 * a[4] * cp.power(y_elig[2] + y_inel[1], 5)
func += 1/4 * a[3] * cp.power(y_elig[2] + y_inel[1], 4)
func += 1/3 * a[2] * cp.power(y_elig[2] + y_inel[1], 3)
func += 1/2 * a[1] * cp.power(y_elig[2] + y_inel[1], 2)
func += a[0] * (y_elig[2] + y_inel[1])

objective = cp.Minimize(func)


# Constraints:

constraints = []
constraints += [y_elig >= 0, y_inel >= 0]
constraints += [cp.sum(y_elig) == 1, cp.sum(y_inel) == 1]
constraints += [y_elig[1] * tau <= B]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
print("y_elig.value:", np.round(y_elig.value, 4) )
print("y_inel.value:", np.round(y_inel.value, 4) )
print()



# power(x, p)



In [ ]:
x1 = 1
x2 = 2
print("x1:", x1, ", x2:", x2)

## CVXPY can handle 4d arrays:

In [ ]:

I, J, K, L = 2, 3, 4, 5

# Variables:
x_test = {}
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                x_test[(i, j, k, ell)] = cp.Variable(1)
            
# Objective:

func = 0.0
func += cp.sum([x_test[(i, j, k, ell)]**2 for i in range(I) for j in range(J) \
                for k in range(K) for ell in range(L)])
            
objective = cp.Minimize(func)

# Constraints:
constraints = []

for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                constraints += [cp.sum([x_test[(i, j, k, ell)] for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L) ]) == 1.0]
                constraints += [x_test[(i, j, k, ell)] >= 0.0 for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L)]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                print("i, j, k, ell:", i, j, k, ell)
                print("x_test[(i,j,k, ell)].value:", x_test[(i, j, k, ell)].value)
